# Application de prédiction automatique de la validité des billets
Ce notebook permet de prédire si un billet est vrai ou faux à partir d’un fichier CSV, en utilisant le modèle Random Forest entraîné.


# 1. Import des bibliothèques


In [ ]:
import pandas as pd
import numpy as np
import io
from IPython.display import display
import ipywidgets as widgets
import joblib


# 2. Chargement du modèle Random Forest et du scaler


In [ ]:
rf = joblib.load('model_rf.joblib')
scaler = joblib.load('scaler.joblib')


# 3. Import du fichier CSV via un bouton interactif


In [ ]:
upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False
)

def handle_upload(change):
    uploaded_file = list(upload_widget.value.values())[0]
    content = uploaded_file['content']
    df_imported = pd.read_csv(io.BytesIO(content))

    global df_test
    df_test = df_imported
    display(df_test.head())

upload_widget.observe(handle_upload, names='value')
display(upload_widget)


# 4. Prédiction et affichage des résultats


In [ ]:
# Colonnes utilisées dans le modèle
colonnes_modele = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']

# Normalisation
df_test_scaled = scaler.transform(df_test[colonnes_modele])

# Prédictions
predictions = rf.predict(df_test_scaled)
probas = rf.predict_proba(df_test_scaled)

# Ajout des résultats
df_test['is_genuine'] = predictions
df_test['proba_faux'] = probas[:, 0].round(3)
df_test['proba_vrai'] = probas[:, 1].round(3)

# Affichage final
colonnes_affichees = colonnes_modele + ['is_genuine', 'proba_faux', 'proba_vrai']
display(df_test[['id'] + colonnes_affichees].head(10))

